# Parse Mediapart article to produce a map plot

## Article source

https://www.mediapart.fr/journal/politique/020724/legislatives-la-galerie-des-candidats-rn-injustement-salis-selon-bardella?userid=d599962d-c199-4f52-af8b-3aca7df6b63b

## Temp file
We cannot directly access this URL as it required a paid account.  
We save the HTML as a temp file bearing such name and path:
`/content/Législatives _ la galerie des candidats RN injustement salis selon Bardella _ Mediapart.html`

## install parser

In [63]:
!pip install html5lib

## parse local HTML file

In [65]:
from bs4 import BeautifulSoup

def parse_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html5lib')

    data = []

    for li in soup.find_all('li', {'data-categories': True}):
        category = li['data-categories']
        img_tag = li.find('img')
        img_src = img_tag['src'] if img_tag else ''
        figcaption = li.find('figcaption')

        # Extracting the name
        name_tag = figcaption.find('strong')
        name = name_tag.text.strip() if name_tag else ''

        # Extracting the subtitle
        subtitle_tag = figcaption.find('p', class_='subtitle')
        subtitle = subtitle_tag.text.strip() if subtitle_tag else ''

        # Extracting the description
        description_tags = figcaption.find_all('p')
        description = ' '.join(p.text.strip() for p in description_tags[2:]) if len(description_tags) > 2 else ''

        data.append({
            'category': category,
            'image_src': img_src,
            'name': name,
            'subtitle': subtitle,
            'description': description
        })

    return data


# Example usage
file_path = '/content/Législatives _ la galerie des candidats RN injustement salis selon Bardella _ Mediapart.html'
parsed_data = parse_html(file_path)
for item in parsed_data:
    print(item)


{'category': 'Ain', 'image_src': 'https://infographics.mediapart.fr/custom-pages/candidats-rn/mourey.jpg', 'name': 'Jean-Marie Mourey', 'subtitle': '1ère circonscription de l’Ain - 39,37\xa0% Suppléant', 'description': '« Vive la France, vive le roi\xa0!\xa0» Sur ses réseaux sociaux –\xa0aujourd’hui désactivés\xa0–, le candidat RN a longtemps partagé les publications de l’Action française (AF), mouvement royaliste dont il fut un responsable départemental pendant de nombreuses années. En 2019, Jean-Marie Mourey faisait part de son refus de participer aux élections, «\xa0ne pouvant cautionner cette République par [son] vote\xa0», associant la Révolution française au «\xa0vol\xa0», au «\xa0viol\xa0» et au «\xa0crime\xa0». Auprès de Mediapart, le candidat affirme avoir démissionné de l’AF en rejoignant le RN en 2022.'}
{'category': 'Ain', 'image_src': 'https://infographics.mediapart.fr/custom-pages/candidats-rn/buisson.jpg', 'name': 'Jérôme Buisson', 'subtitle': '4ème circonscription de l’

## Build dataframe from parsed data

In [71]:
import pandas as pd

parsed_data_df = pd.DataFrame(parsed_data)
parsed_data_df.columns = ['departement_name', 'image_src', 'candidate_name', 'candidate_score', 'description']
parsed_data_df

,departement_name,image_src,candidate_name,candidate_score,description
0,Ain,https://infographics.mediapart.fr/custom-pages...,Jean-Marie Mourey,"1ère circonscription de l’Ain - 39,37 % Suppléant","« Vive la France, vive le roi ! » Sur ses rése..."
1,Ain,https://infographics.mediapart.fr/custom-pages...,Jérôme Buisson,"4ème circonscription de l’Ain - 46,01 % Sortant",À une déclaration du député noir Carlos Marten...
2,Aisne,https://infographics.mediapart.fr/custom-pages...,Jocelyn Dessigny,"5ème circonscription de l’Aisne - 53,07 % Sort...","Sur son compte Facebook, le député et candidat..."
3,Allier,https://infographics.mediapart.fr/custom-pages...,Jorys Bovet,"2ème circonscription de l’Allier - 34,33 % Sor...",Le député se saisit régulièrement de faits div...
4,Alpes-Maritimes,https://infographics.mediapart.fr/custom-pages...,Lionel Tivoli,"2ème circonscription des Alpes-Maritimes - 48,...",Si Lionel Tivoli fait désormais campagne avec ...
...,...,...,...,...,...
67,Yonne,https://static.mediapart.fr/etmagine/default/f...,Daniel Grenon,"1ère circonscription de l’Yonne - 40,40 % Sortant",Daniel Grenon n’aime que l’on dise qu’il tient...
68,Essonne,https://static.mediapart.fr/etmagine/default/f...,Thiebauld Vega,"1ère circonscription de l’Essonne - 22,34 %","Obsédé par les musulmans et le porc, il propos..."
69,Essonne,https://static.mediapart.fr/etmagine/default/f...,Jérôme Carbriand,"4ème circonscription de l’Essonne - 31,03 %","De 2012 à 2014, le candidat RN a alimenté un b..."
70,Val-d’Oise,https://infographics.mediapart.fr/custom-pages...,Anne Sicard,"1ère circonscription du Val-d’Oise - 33,65 %",En tête dans le Val-d’Oise après le premier to...


## Get circo dataset

### Get circo positions dataset (GEOJSON)

In [100]:
import geopandas as gpd

circo_coords = gpd.read_file('https://static.data.gouv.fr/resources/contours-geographiques-des-circonscriptions-legislatives/20240613-191520/circonscriptions-legislatives-p10.geojson')
circo_coords.head()

,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,geometry
0,01,Ain,0104,4ème circonscription,"POLYGON ((4.92750 45.98000, 4.92450 45.96820, ..."
1,01,Ain,0105,5ème circonscription,"POLYGON ((5.59060 45.76000, 5.59040 45.76470, ..."
2,01,Ain,0103,3ème circonscription,"POLYGON ((5.58640 45.66710, 5.58730 45.65990, ..."
3,01,Ain,0102,2ème circonscription,"POLYGON ((4.74920 46.00360, 4.74850 46.00200, ..."
4,01,Ain,0101,1ère circonscription,"POLYGON ((5.51130 46.26450, 5.50840 46.26440, ..."


### create merge index on Mediapart data

In [114]:
def extract_circo_name(circo_and_score):
  split_text = circo_and_score.split(' ')
  return split_text[0] +' '+ split_text[1]


parsed_data_df['circo_name'] = parsed_data_df['candidate_score'].apply(extract_circo_name)
parsed_data_df['merge_index'] = parsed_data_df['departement_name'] + '-' + parsed_data_df['circo_name']
parsed_data_df['merge_index'] = parsed_data_df['merge_index'].str.replace('’', "'")
parsed_data_df.head()

,departement_name,image_src,candidate_name,candidate_score,description,merge_index,circo_name
0,Ain,https://infographics.mediapart.fr/custom-pages...,Jean-Marie Mourey,"1ère circonscription de l’Ain - 39,37 % Suppléant","« Vive la France, vive le roi ! » Sur ses rése...",Ain-1ère circonscription,1ère circonscription
1,Ain,https://infographics.mediapart.fr/custom-pages...,Jérôme Buisson,"4ème circonscription de l’Ain - 46,01 % Sortant",À une déclaration du député noir Carlos Marten...,Ain-4ème circonscription,4ème circonscription
2,Aisne,https://infographics.mediapart.fr/custom-pages...,Jocelyn Dessigny,"5ème circonscription de l’Aisne - 53,07 % Sort...","Sur son compte Facebook, le député et candidat...",Aisne-5ème circonscription,5ème circonscription
3,Allier,https://infographics.mediapart.fr/custom-pages...,Jorys Bovet,"2ème circonscription de l’Allier - 34,33 % Sor...",Le député se saisit régulièrement de faits div...,Allier-2ème circonscription,2ème circonscription
4,Alpes-Maritimes,https://infographics.mediapart.fr/custom-pages...,Lionel Tivoli,"2ème circonscription des Alpes-Maritimes - 48,...",Si Lionel Tivoli fait désormais campagne avec ...,Alpes-Maritimes-2ème circonscription,2ème circonscription


There is no unicity, which means we will get some duplicates, we will adress this later!

In [115]:
len(parsed_data_df) == len(parsed_data_df['merge_index'].unique())

False

### Create merge index on coordinates dataset

In [116]:
circo_coords['merge_index'] = circo_coords['nomDepartement'] + '-' + circo_coords['nomCirconscription']

In [117]:
circo_coords.head()

,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,geometry,merge_index
0,01,Ain,0104,4ème circonscription,"POLYGON ((4.92750 45.98000, 4.92450 45.96820, ...",Ain-4ème circonscription
1,01,Ain,0105,5ème circonscription,"POLYGON ((5.59060 45.76000, 5.59040 45.76470, ...",Ain-5ème circonscription
2,01,Ain,0103,3ème circonscription,"POLYGON ((5.58640 45.66710, 5.58730 45.65990, ...",Ain-3ème circonscription
3,01,Ain,0102,2ème circonscription,"POLYGON ((4.74920 46.00360, 4.74850 46.00200, ...",Ain-2ème circonscription
4,01,Ain,0101,1ère circonscription,"POLYGON ((5.51130 46.26450, 5.50840 46.26440, ...",Ain-1ère circonscription


In [118]:
circo_coords[circo_coords.nomDepartement.str.contains('Or')]

,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,geometry,merge_index
84,21,Côte-d'Or,2105,5ème circonscription,"MULTIPOLYGON (((5.17610 47.15360, 5.17170 47.1...",Côte-d'Or-5ème circonscription
85,21,Côte-d'Or,2104,4ème circonscription,"POLYGON ((4.29280 47.92590, 4.29670 47.92300, ...",Côte-d'Or-4ème circonscription
86,21,Côte-d'Or,2101,1ère circonscription,"POLYGON ((4.84180 47.27890, 4.84780 47.27920, ...",Côte-d'Or-1ère circonscription
87,21,Côte-d'Or,2103,3ème circonscription,"POLYGON ((5.25560 47.17600, 5.25960 47.17840, ...",Côte-d'Or-3ème circonscription
88,21,Côte-d'Or,2102,2ème circonscription,"POLYGON ((5.25560 47.17600, 5.25770 47.17550, ...",Côte-d'Or-2ème circonscription
302,61,Orne,6101,1ère circonscription,"POLYGON ((-0.86030 48.50150, -0.85880 48.50040...",Orne-1ère circonscription
303,61,Orne,6103,3ème circonscription,"POLYGON ((-0.84120 48.75240, -0.84230 48.75130...",Orne-3ème circonscription
304,61,Orne,6102,2ème circonscription,"POLYGON ((0.06360 48.91670, 0.06480 48.91650, ...",Orne-2ème circonscription
330,66,Pyrénées-Orientales,6604,4ème circonscription,"POLYGON ((2.93280 42.47440, 2.93480 42.47440, ...",Pyrénées-Orientales-4ème circonscription
331,66,Pyrénées-Orientales,6602,2ème circonscription,"POLYGON ((2.32290 42.70810, 2.33170 42.70760, ...",Pyrénées-Orientales-2ème circonscription


In [120]:
# 'Côte-d'Or-4ème circonscription' == 'Côte-d’Or-4ème circonscription'

### add candidate info to mediapart info

In [121]:
circo_dataset = pd.merge(parsed_data_df, circo_coords, on='merge_index', how='left')

Check merge errors: nothing to declare.

In [122]:
circo_dataset[circo_dataset['codeCirconscription'].isnull()]

,departement_name,image_src,candidate_name,candidate_score,description,merge_index,circo_name,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,geometry


## Add info to leaflet map!

In [129]:
# Simplify the process and ensure GeoJSON structure

import folium
from shapely.geometry import shape
import json

# Initialize a map centered around France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Function to create tooltip
# Function to create tooltip
def create_tooltip(row):
    tooltip_html = f"""
    <div style="width: 200px; white-space: normal; word-wrap: break-word;">
        <img src="{row['image_src']}" alt="Candidate Image" style="width:100%; height:auto;">
        <h4>{row['candidate_name']}</h4>
        <h5>{row['candidate_score']}</h5>
        <p>{row['description']}</p>
        <p style="color:gray;">{row['nomDepartement']} - {row['nomCirconscription']}</p>
    </div>
    """
    return tooltip_html

# Add markers to the map
for _, row in circo_dataset.iterrows():

    geojson = row['geometry']

    # Add the GeoJSON object to the map
    folium.GeoJson(
        geojson,
        tooltip=folium.Tooltip(create_tooltip(row)),
        popup=folium.Popup(create_tooltip(row), max_width=300)
    ).add_to(m)

# Save the map to an HTML file
map_file_path = 'candidates_map.html'
m.save(map_file_path)
m

## Solve for multiple people for a single circo

We display side-by-side in tooltip and marker.  

In [152]:
import folium
from shapely.geometry import shape
import json
import pandas as pd
from collections import defaultdict

# Initialize a map centered around France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Function to create tooltip
def create_tooltip(row):
    tooltip_html = f"""
    <div style="flex: 1; min-width: 50%; box-sizing: border-box; padding: 5px; white-space: normal; word-wrap: break-word;">
        <img src="{row['image_src']}" alt="Candidate Image" style="width:100%; height:auto;">
        <h4>{row['candidate_name']}</h4>
        <h5>{row['candidate_score']}</h5>
        <p>{row['description']}</p>
        <p style="color:gray;">{row['nomDepartement']} - {row['nomCirconscription']}</p>
    </div>
    """
    return tooltip_html

# Group rows by their geometry to handle duplicates
grouped = circo_dataset.groupby('geometry')

for geojson, group in grouped:
    tooltip_content = ""

    # If more than one row refers to the same place, create a dual tooltip
    tooltip_content += f'<div style="display: flex; flex-wrap: wrap; width: {len(group)*200}px;">'
    for _, row in group.iterrows():
        tooltip_content += create_tooltip(row)
    tooltip_content += '</div>'

    geojson_obj = row['geometry']

    # Add the GeoJSON object to the map with combined tooltip content
    folium.GeoJson(
        geojson_obj,
        tooltip=folium.Tooltip(tooltip_content),
        popup=folium.Popup(tooltip_content, max_width=600)
    ).add_to(m)

# Save the map to an HTML file
map_file_path = 'candidates_map.html'
m.save(map_file_path)
m


## Add outrage categories  

What are these people accused of?  

We submit the list to ChatGPT and ask for 5 possible tags to categorize each description (each description has only one tag).

In [158]:
circo_dataset[['candidate_name', 'description']].to_csv('export_2.csv')

In [173]:
accusation_category = """
"candidate_name","accusation_category"
"Jean-Marie Mourey","Racisme/Xénophobie"
"Jérôme Buisson","Racisme/Xénophobie"
"Jocelyn Dessigny","Racisme/Xénophobie"
"Jorys Bovet","Racisme/Xénophobie"
"Lionel Tivoli","Islamophobie"
"Andrea Orabona","Théories Conspirationnistes/Nostalgie Fasciste"
"Bryan Masson","Racisme/Xénophobie"
"Thierry Dussud","Racisme/Xénophobie"
"Philippe Arbona","Théories Conspirationnistes/Nostalgie Fasciste"
"Monique Griseti","Théories Conspirationnistes/Nostalgie Fasciste"
"Joëlle Mélin","Théories Conspirationnistes/Nostalgie Fasciste"
"Ludivine Daoudi","Théories Conspirationnistes/Nostalgie Fasciste"
"Josseline Liban","Théories Conspirationnistes/Nostalgie Fasciste"
"Philippe Chapron","Antisémitisme/Négationnisme"
"Pierre Gentillet","Théories Conspirationnistes/Nostalgie Fasciste"
"Julie Apricena","Théories Conspirationnistes/Nostalgie Fasciste"
"Maïtey Pouget","Racisme/Xénophobie"
"Valéry Élophe","Racisme/Xénophobie"
"Sophie Dumont","Racisme/Xénophobie"
"René Lioret","Racisme/Xénophobie"
"Françoise Billaud","Racisme/Xénophobie"
"Noël Lude","Racisme/Xénophobie"
"Florence Joubert","Racisme/Xénophobie"
"Thibaut Monnier","Racisme/Xénophobie"
"Tony Bihouée","Racisme/Xénophobie"
"Christian Perez","Racisme/Xénophobie"
"Nicolas Meizonnet","Racisme/Xénophobie"
"Pascale Bordes","Islamophobie"
"Frank Khalifa","Islamophobie"
"Stéphanie Alarcon","Racisme/Xénophobie"
"Sandrine Chadourne","Racisme/Xénophobie"
"Jean-Pierre Mareschi","Racisme/Xénophobie"
"Dylan Lemoine","Racisme/Xénophobie"
"Mylène Wunsch","Sexisme/Homophobie/Violences Verbales"
"Corine Fougeron","Théories Conspirationnistes/Nostalgie Fasciste"
"Thierry Mosca","Corruption ou Délits Financiers"
"Ludovic Biesbrouck","Racisme/Xénophobie"
"Roger Chudeau","Racisme/Xénophobie"
"Gerbert Rambaud","Sexisme/Homophobie/Violences Verbales"
"Michel Lucas","Théories Conspirationnistes/Nostalgie Fasciste"
"Bruno Comby","Théories Conspirationnistes/Nostalgie Fasciste"
"Anthony Zeller","Théories Conspirationnistes/Nostalgie Fasciste"
"Jean-Pierre Templier","Antisémitisme/Négationnisme"
"Gilles Bourdouleix","Antisémitisme/Négationnisme"
"Paule Veyre de Soras","Racisme/Xénophobie"
"Jean-Michel Cadenas","Théories Conspirationnistes/Nostalgie Fasciste"
"Annie Bell-Jaccoud","Théories Conspirationnistes/Nostalgie Fasciste"
"Louis-Joseph Pecher","Antisémitisme/Négationnisme"
"Joseph Martin","Antisémitisme/Négationnisme"
"Florent de Kersauson","Corruption ou Délits Financiers"
"Gregory Renard","Théories Conspirationnistes/Nostalgie Fasciste"
"Laurent Jacobelli","Islamophobie"
"Monique Becker","Théories Conspirationnistes/Nostalgie Fasciste"
"Sylviane Lopez","Racisme/Xénophobie"
"Marie-Christine Sorin","Racisme/Xénophobie"
"Olivier Monteil","Islamophobie"
"Nathalie Aubert","Racisme/Xénophobie"
"Brice Bernard","Antisémitisme/Négationnisme"
"Ivanka Dimitrova","Racisme/Xénophobie"
"Jean-Yves Le Boulanger","Racisme/Xénophobie"
"Teddy Robin","Sexisme/Homophobie/Violences Verbales"
"Gaëtan Brault","Sexisme/Homophobie/Violences Verbales"
"Dorothée Champeau","Racisme/Xénophobie"
"Olivier Guibert","Théories Conspirationnistes/Nostalgie Fasciste"
"Philippe Lottiaux","Racisme/Xénophobie"
"Philippe Schreck","Islamophobie"
"Emmanuelle Darles","Théories Conspirationnistes/Nostalgie Fasciste"
"Daniel Grenon","Racisme/Xénophobie"
"Thiebauld Vega","Islamophobie"
"Jérôme Carbriand","Antisémitisme/Négationnisme"
"Anne Sicard","Théories Conspirationnistes/Nostalgie Fasciste"
"Nadejda Remy","Théories Conspirationnistes/Nostalgie Fasciste"
"""

In [174]:
import io

accusation_category_df = pd.read_csv(io.StringIO(accusation_category), sep=",")
accusation_category_df

,candidate_name,accusation_category
0,Jean-Marie Mourey,Racisme/Xénophobie
1,Jérôme Buisson,Racisme/Xénophobie
2,Jocelyn Dessigny,Racisme/Xénophobie
3,Jorys Bovet,Racisme/Xénophobie
4,Lionel Tivoli,Islamophobie
...,...,...
67,Daniel Grenon,Racisme/Xénophobie
68,Thiebauld Vega,Islamophobie
69,Jérôme Carbriand,Antisémitisme/Négationnisme
70,Anne Sicard,Théories Conspirationnistes/Nostalgie Fasciste


In [175]:
circo_dataset_with_accusations = pd.merge(circo_dataset,accusation_category_df, on='candidate_name', how='left')

In [176]:
circo_dataset_with_accusations[circo_dataset_with_accusations['accusation_category'].isnull()]

,departement_name,image_src,candidate_name,candidate_score,description,merge_index,circo_name,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,geometry,accusation_category


In [177]:
circo_dataset_with_accusations.accusation_category.value_counts()

accusation_category
Racisme/Xénophobie                                32
Théories Conspirationnistes/Nostalgie Fasciste    20
Islamophobie                                       7
Antisémitisme/Négationnisme                        7
Sexisme/Homophobie/Violences Verbales              4
Corruption ou Délits Financiers                    2
Name: count, dtype: int64

## Update map to color places based on accusation category and add legend

In [191]:
import folium
from shapely.geometry import shape
import json
import pandas as pd
from collections import defaultdict

# Initialize a map centered around France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Function to create tooltip
def create_tooltip(row):
    tooltip_html = f"""
    <div style="flex: 1; min-width: 50%; box-sizing: border-box; padding: 5px; white-space: normal; word-wrap: break-word;">
        <img src="{row['image_src']}" alt="Candidate Image" style="width:100%; height:auto;">
        <h4>{row['candidate_name']}</h4>
        <h5>{row['candidate_score']}</h5>
        <p>{row['description']}</p>
        <p style="color:gray;">{row['nomDepartement']} - {row['nomCirconscription']}</p>
    </div>
    """
    return tooltip_html

# Define color mapping for accusation categories
color_mapping = {
    'Racisme/Xénophobie': 'red',
    'Théories Conspirationnistes/Nostalgie Fasciste': 'blue',
    'Islamophobie': 'green',
    'Antisémitisme/Négationnisme': 'purple',
    'Sexisme/Homophobie/Violences Verbales': 'orange',
    'Corruption ou Délits Financiers': 'yellow'
}

# Group rows by their geometry to handle duplicates
grouped = circo_dataset_with_accusations.groupby('geometry')

for geojson, group in grouped:
    tooltip_content = ""

    # If more than one row refers to the same place, create a dual tooltip
    tooltip_content += f'<div style="display: flex; flex-wrap: wrap; width: {len(group)*200}px;">'
    for _, row in group.iterrows():
        tooltip_content += create_tooltip(row)
    tooltip_content += '</div>'

    geojson_obj = row['geometry']
    accusation_category = row['accusation_category']
    color = color_mapping.get(accusation_category, 'gray')  # Default to gray if category is not found

    # Add the GeoJSON object to the map with combined tooltip content
    folium.GeoJson(
        geojson_obj,
        tooltip=folium.Tooltip(tooltip_content),
        popup=folium.Popup(tooltip_content, max_width=600),
        style_function=lambda x, color=color: {'fillColor': color, 'color': 'black', 'weight': 1, 'fillOpacity': 0.5}
    ).add_to(m)

# Add legend to the map
legend_html = """
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 300px; height: 300px;
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; border-radius:6px; padding: 10px;">
    <h4 style="margin-top:0;">Legende</h4>
    <p><span style="background-color: red; width: 20px; height: 20px; display: inline-block;"></span> Racisme/Xénophobie</p>
    <p><span style="background-color: blue; width: 20px; height: 20px; display: inline-block;"></span> Théories Conspirationnistes/Nostalgie Fasciste</p>
    <p><span style="background-color: green; width: 20px; height: 20px; display: inline-block;"></span> Islamophobie</p>
    <p><span style="background-color: purple; width: 20px; height: 20px; display: inline-block;"></span> Antisémitisme/Négationnisme</p>
    <p><span style="background-color: orange; width: 20px; height: 20px; display: inline-block;"></span> Sexisme/Homophobie/Violences Verbales</p>
    <p><span style="background-color: yellow; width: 20px; height: 20px; display: inline-block;"></span> Corruption ou Délits Financiers</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
map_file_path = 'candidates_map.html'
m.save(map_file_path)
m
